In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import json
import numpy as np
import os

In [5]:
root_path = "exp"
model_names = ["bi", "dtu", "utd"]

In [6]:
result_df = pd.DataFrame(columns=["model", "cv", "acc", "precision", "recall", "f1"])
for model_name in model_names:
    for cv in range(5):
        exp_path = os.path.join(root_path, f"{model_name}_cv{cv}")
        eval_df = pd.read_csv(os.path.join(exp_path, "eval.csv"))
        max_idx = eval_df['acc'].argmax()
        acc, precision, recall, f1 = eval_df['acc'].iloc[max_idx], eval_df['precision'].iloc[max_idx], eval_df['recall'].iloc[max_idx], eval_df['f1'].iloc[max_idx]
        result_df = result_df.append({"model": model_name, "cv": cv, "acc": acc, 
                                      "precision": precision, "recall": recall, "f1": f1}, ignore_index=True)

In [7]:
result_df

,model,cv,acc,precision,recall,f1
0,bi,0,0.7659,0.7064,0.7659,0.713
1,bi,1,0.7431,0.6862,0.7431,0.6903
2,bi,2,0.7467,0.6941,0.7467,0.6949
3,bi,3,0.7435,0.6865,0.7435,0.6899
4,bi,4,0.7467,0.6903,0.7467,0.6952
5,dtu,0,0.5142,0.3887,0.5142,0.4402
6,dtu,1,0.5796,0.52,0.5796,0.5108
7,dtu,2,0.583,0.5234,0.583,0.514
8,dtu,3,0.5884,0.5211,0.5884,0.5164
9,dtu,4,0.5911,0.5168,0.5911,0.5178


In [8]:
result_df.groupby("model").mean()

,cv,acc,precision,recall,f1
model,,,,,
bi,2.0,0.74918,0.6927,0.74918,0.69666
dtu,2.0,0.57126,0.4940,0.57126,0.49984
utd,2.0,0.62242,0.4766,0.62242,0.52988


# Table for paper

In [14]:
branches = ["LMA", "LAD", "LCX", "D", "OM", "weighted avg"]
model_names = ["utd", "dtu", "bi"]
for model_name in model_names:
    acc, prec, rec, f1 = {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}
    for cv in range(5):
        exp_path = os.path.join(root_path, f"{model_name}_cv{cv}")
        eval_df = pd.read_csv(os.path.join(exp_path, "eval.csv"))
        max_idx = eval_df['acc'].argmax()
        epoch = eval_df['epoch'].loc[max_idx]
        print("---------------", epoch)
        clf_report = json.load(open(os.path.join(exp_path, f"clf_report_{epoch:04d}.json"), "r"))
        cm = np.load(os.path.join(exp_path, f"confusion_matrix_{epoch:04d}.npy"))
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print(cm.diagonal())
        for idx, k in enumerate(branches):
            if idx < len(branches) - 1:
                acc[k].append(cm.diagonal()[idx])
            else:
                acc[k].append(clf_report["accuracy"])
            prec[k].append(clf_report[k]['precision'])
            rec[k].append(clf_report[k]['recall'])
            f1[k].append(clf_report[k]['f1-score'])
    table_df = pd.DataFrame(columns=["metric"]+branches)
    row_data = {"metric": "acc"}
    for k in branches:
        row_data.update({k: f"{np.mean(acc[k]):0.4f}±{np.std(acc[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    row_data = {"metric": "pre"}
    for k in branches:
        row_data.update({k: f"{np.mean(prec[k]):0.4f}±{np.std(prec[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    row_data = {"metric": "rec"}
    for k in branches:
        row_data.update({k: f"{np.mean(rec[k]):0.4f}±{np.std(rec[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    row_data = {"metric": "f1"}
    for k in branches:
        row_data.update({k: f"{np.mean(f1[k]):0.4f}±{np.std(f1[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    table_df.to_csv(f"{model_name}.csv", index=False)

--------------- 10
--------------- 30
--------------- 10
--------------- 10
--------------- 10
--------------- 30
--------------- 30
--------------- 30
--------------- 30
--------------- 110
--------------- 10
--------------- 10
--------------- 10
--------------- 30
--------------- 60


In [10]:
# table for AAAI
root_path = "exp_aaai"
branches = ["LMA", "LAD", "LCX", "D", "OM", "weighted avg"]
model_names = ["bi"]
for model_name in model_names:
    acc, prec, rec, f1 = {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}
    for cv in range(5):
        exp_path = os.path.join(root_path, f"{model_name}_cv{cv}")
        eval_df = pd.read_csv(os.path.join(exp_path, "eval.csv"))
        max_idx = eval_df['acc'].argmax()
        epoch = eval_df['epoch'].loc[max_idx]
        print("---------------", epoch)
        clf_report = json.load(open(os.path.join(exp_path, f"clf_report_{epoch:04d}.json"), "r"))
        cm = np.load(os.path.join(exp_path, f"confusion_matrix_{epoch:04d}.npy"))
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print(cm.diagonal())
        for idx, k in enumerate(branches):
            if idx < len(branches) - 1:
                acc[k].append(cm.diagonal()[idx])
            else:
                acc[k].append(clf_report["accuracy"])
            prec[k].append(clf_report[k]['precision'])
            rec[k].append(clf_report[k]['recall'])
            f1[k].append(clf_report[k]['f1-score'])
    table_df = pd.DataFrame(columns=["metric"]+branches)
    row_data = {"metric": "acc"}
    for k in branches:
        row_data.update({k: f"{np.mean(acc[k]):0.4f}±{np.std(acc[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    row_data = {"metric": "pre"}
    for k in branches:
        row_data.update({k: f"{np.mean(prec[k]):0.4f}±{np.std(prec[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    row_data = {"metric": "rec"}
    for k in branches:
        row_data.update({k: f"{np.mean(rec[k]):0.4f}±{np.std(rec[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    row_data = {"metric": "f1"}
    for k in branches:
        row_data.update({k: f"{np.mean(f1[k]):0.4f}±{np.std(f1[k]):0.4f}"})
    table_df = table_df.append(row_data, ignore_index=True)
    table_df.to_csv(f"{model_name}_aaai.csv", index=False)

--------------- 10
--------------- 10
--------------- 10
--------------- 150
--------------- 10


In [7]:
exp_path = os.path.join(root_path, f"{model_name}_cv{cv}")
print(exp_path)
eval_df = pd.read_csv(os.path.join(exp_path, "eval.csv"))
eval_df

exp_aaai/bi_cv4


,epoch,acc,precision,recall,f1
